# Loss vs Wavelength in Glass Tube

This uses the 3 layer exact RadialStep class and 2 layer StepExact class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from step_exact import RadialStep
from scipy.optimize import newton

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
betas = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

# Three Layer Study: Air Outside Cladding

In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = RadialStep(r1=15*2.7183333333333333e-6, t1=15*2/3*1e-6, t2=15*.4e-6, 
                   scale=15e-6, R1_maxh=5, R2_maxh=5, R3_maxh=5, wavelength=wl)

    k_low = A.k1 * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.hybrid_det_matrix, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.9999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.hybrid_det_matrix(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .99999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas[i] = beta
    

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.n1  # Inner (core) index
n2 = A.n2  # Cladding index

d = 15*2/3*1e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas.imag, color='green', linewidth=.9)
for l in ls:
    plt.ylim(-.001, .012)
    plt.plot([l,l], [0, .012],  linewidth=1, linestyle=':')

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, betas.real, color='green', linewidth=.9)


In [ ]:
np.save('exact_scaled_betas', betas)